# Hjelp jeg prøvde å bruke Fully Homomorphic Encryption (FHE)

* Homomorphic operations
* Historie
* Fully Homomorphic operations
* Mitt forsøk på å bruke FHE

### Hva er kryptering?
* Lagre data
* Sende data

* Behandle data?

![Screenshot 2023-02-22 164309.png](Screenshot_2023-02-22_164309.png)

I 1978 lurte 3 smarte menn fra MIT på hvordan en fiktiv bank kunne bruke en server (time-shared service) til å kalkulere sensitive data fra sine låntakere uten å gi fra seg noe informasjon. Et av forslagene var å bruke homomorfisk kryptering.

Dette var kun teoretisk i 30 år før Gentry laget det først FHE rammeverket i 2009. I 2011 kom Microsoft ut med et paper “Can Homomorphic Encryption be Practical?”. I 2018 kom Microsoft ut med en implementasjon kalt Simple Encrypted Arithmetic Library (SEAL). Og i 2022 kom Edge med Password Monitor som bruker FHE til å sjekke om passordet ditt finnes i lekkede passord-databaser. (Chrome kom kort tid etterpå med en hash-basert løsning).


# Hva er Homomorphic Encryption?
* Anna har en melding $m$
* Anna krypterer $m$. $\text{enc}(m)$.
* Anna sender $\text{enc}(m)$ til server.
* Serveren gjør en funksjon $f()$ på den krypterte meldingen $\text{enc}(f(m))$.
* Serveren sender krypterte resultatet $\text{enc}(f(m))$ tilbake.
* Anna kan dekryptere svaret. $\text{dec}((\text{enc}(f(m))) = f(m)$.

### RSA
1. Bob selects two large prime $p<<0$ and $q<<0$ and calclate $n=p \times q$
2. Bob calculate $\phi(n) = (p-1)(p+2)$. Choose $e$ such that $\gcd(e, \phi(n))=1$.
3. Bob calculate private key $d$ to be the modular inverse of $e$, in other words $de \equiv 1 (\mod \phi(n))$.
4. Bob sends the public key $p_k = n,p$
5. Anna encrypts message $m$ to ciphertext $c \equiv m^e (\mod n)$. And sends it.
6. Bob computes the message $c^d \equiv m (\mod n)$

Før FHE RSA

In [1]:
from math import gcd
# Bobs computer
p, q, e = 11, 17, 3 # pretend that p and 1 are large (4096 bits)
n = p * q #187
print(f'{n=}')
phi_n = (p-1)*(q-1)
print(f'{gcd(e, phi_n)}')
d = pow(e, -1, mod = phi_n)

# Annas computer
message_sent = 42
cipher = message_sent**e % n
print(f'{cipher=}')

# Bobs computer
message_recieved = cipher**d % n
print(f'{message_recieved=}, {message_recieved==message_sent=}')


n=187
1
cipher=36
message_recieved=42, message_recieved==message_sent=True



### Multiplicative homomorphism in RSA

In [1]:
from math import gcd
# Bobs computer
p, q, e = 11, 17, 3
#p, q = 101, 103
#e = 7
n = p * q #187
print(f'{n=}')
phi_n = (p-1)*(q-1)
print(f'{gcd(e, phi_n)=}')
d = pow(e, -1, mod = phi_n)

# Annas computer
message_sent_j = 10
message_sent_k = 42
message_product = message_sent_j * message_sent_k # Only for verification
cipher_j = message_sent_j**e % n
cipher_k = message_sent_k**e % n
print(f'{(cipher_j,cipher_k)=}')

#Homomorphic calculation
cipher_product = cipher_k * cipher_j

# Bobs computer
message_recieved = cipher_product**d % n
print(f'{(message_recieved, message_recieved==message_product, message_recieved==message_product%n)=}')

n=187
gcd(e, phi_n)=1
(cipher_j,cipher_k)=(65, 36)
(message_recieved, message_recieved==message_product, message_recieved==message_product%n)=(46, False, True)


### Takebacks
Ikke evig multiplikasjoner, `ciper_product`$ \equiv  m_j \times m_j \bmod {n}$

## Paillier

### Generate keys
$$
\begin{align}
\mathbb{Z}_n &= \{0,1,2, \ldots, n-1\} \\
p,q &\in \mathbb{N}_{p(rime)} | \gcd ( pq, (p−1)(q−1))=1 \\
n &=pq\\
\lambda &= \text{lcm} (p-1,q-1) \\
g   &\in_R \mathbb{Z}_{n^2}^* \\
L(x)& = {{x-1}\over n} \\
\mu &=(L(g^{\lambda }{\bmod \; n}^{2}))^{{-1}}{\bmod \; n}
b \\
\text{public key} =  k_{pub} &= (n, g)  \\
\text{private key} = k_{priv} &= (\lambda, \mu)  \\
\end{align}
$$

Extended Euclidean Algorithm for å finne invers

$$
\text{\{public key, private_key\}} =  \{k_{pub},k_{priv}\}  =  \{(n, g),(\lambda, \mu)\}
$$
### Encrypt one message
$$
\begin{align}
\text{message} &= m \in \mathbb{Z}_n \\
\text{random seed} &= r \in_R \mathbb{N} | \gcd(r,n)=1 \\
\text{ciphertext} &=c =\mathcal{E}(m) = g^m \times r^n &\bmod\; n^2 \\
\end{align}
$$

### Decrypt
$$
\begin{align}
L(x)& = {{x-1}\over n} \\
m &=L(c^{\lambda} \bmod \; n^2) \times \mu \bmod \; n
\end{align}
$$

Legg merke til at man multipliserer $g^m$ og $r^n$

$$
\text{\{public key, private_key\}} =  \{k_{pub},k_{priv}\}  =  \{(n, g),(\lambda, \mu)\}
$$
### Encrypt two messages
$$
\begin{align}
\mathcal{E}(m_1) &= g^{m_1} r_1^n&\bmod \;n^2 \\
\mathcal{E}(m_2) &= g^{m_2} r_2^n&\bmod \;n^2 \\
\end{align}
$$

### Homomorphic addition of two ciphertexts
$$
\begin{align}
\mathcal{E}(m_1) \times \mathcal{E}(m_2) &= (g^{m_1} r_1^n)\times(g^{m_2} r_2^n) \bmod \;n^2 \\
&= g^{m_1 + m_2} (r_1 \times r_2)^n \bmod\; n^2 &\; r_x = r_1\times r_2 \\
&= \mathcal{E}(m_1 + m_2)\\
\end{align}
$$

Man multipliserer krypterte outputten for å addere message.

### Homomorphic mulitplication of one ciphertext and non-encrypted constant  $k$
$$
\begin{align}
\mathcal{E}(m)            &= g^m r^n\\
\mathcal{E}(m) \times g^k &= g^m r^n \times g^k\\
                          &= g^{m+k} r^n\\
\end{align}
$$

In [2]:
from phe import paillier
public_key, private_key = paillier.generate_paillier_keypair()
messages = [33, 66]
cipher_texts = [public_key.encrypt(x) for x in messages]
cipher_texts.append(cipher_texts[0]+cipher_texts[1])
cipher_texts.append(cipher_texts[0]*10)
decrypted = [private_key.decrypt(x) for x in cipher_texts]
print(f'{decrypted=}')

decrypted=[33, 66, 99, 330]


Notater:


# Hva kan homomorphism brukes til?
* Anonym avstemning
* Anonym treningsdata til K.I.
* Kan kanskje *ikke* brukes der du *ikke* vil ha *mallebility*.
    * Noen hacker din krypterte IOU liste og ganger alle beløp med 2.
* ?? Kanskje ikke så mye mer...?
* ...Men hva om vi fikk til både $\times$ OG $+$ ???

# $\{ \times, + \} \rightarrow \{\wedge, \vee, \oplus, \; \neg \} \Leftrightarrow \{\&,|,\text{^},!\} $

$$
\begin{align}
a,b &\in\{0,1\}\\
a\oplus b & \Leftrightarrow a+b &\bmod 2 \\
a\wedge b & \Leftrightarrow a\times b \\
a \vee b & \Leftrightarrow a + b + (a \times b) &\bmod 2
\end{align}
$$

| $a$ | $b$ |$ a\oplus b$|$a \wedge b$|$a \vee b $|
|:--|:--|:--|:--|:--|
|$a$|$b$|$a+b\bmod 2$|$a\times b$| $a + b + (a \times b) \bmod 2$|
| 0 | 0 |$0+0=0$     |$0 \times 0 = 0$|$0+0+(0 \times0) = 0$|
| 0 | 1 |$0+1=1$     |$0 \times 1 = 0$|$0+1+(0\times 1) =1$|
| 1 | 0 |$1+0=1$     |$1 \times 0 = 0$|$1+0+(1\times 0) =1$|
| 1 | 1 |$1+1= 2 \equiv0\bmod2$|$1 \times 1 = 1$|$1+1+(1\times 1)=3\equiv1 \bmod 2$|



# FHE **F**ully **H**omomorphic **E**ncryption
* Si noe om Noice budget
* Når du har nådd syøt-budsjettet ditt, kan du ta imot tallet ditt, decryptere det, og sende det inn igjen krypter.
* Bootstrapping: Å dekryptere kryptert data kryptert.
* Noe om first generation, second og third..
* 4th generation FHE:  Cheon, Kim, Kim and Song (CKKS)
    * Homomorphic Encryption for Arithmetic of Approximate Numbers 2016, ASIACRYPT 2017
    * Floats.
    * In short, our encoding function is given by:
    * ![Screenshot](CKKS_enc.png)
    * > ...we show that our scheme can be applied to the efficient evaluation of transcendental functions such as multiplicative inverse, exponential function, logistic function and discrete Fourier transform.
    * > The primary open problem is finding way to convert our scheme to a fully homomorphic scheme using bootstrapping.

    

### Historie (ref wiki)
Dette var kun teoretisk i 30 år før Gentry laget det først FHE rammeverket i 2009. I 2011 kom Microsoft ut med et paper “Can Homomorphic Encryption be Practical?”. I 2018 kom Microsoft ut med en implementasjon kalt Simple Encrypted Arithmetic Library (SEAL). Og i 2022 
* 1978
    * Teorisert
* 1st gen
    * Gentry lagde et *somewhoat homomorphic* krypterings-scheme. Dette kunne plausibilt bli FHE ved å *bootstrappe*(2009)- *Bootstrapping er når man kan implementere dekrypteringsalgoritmen i schemet.*
    * Fully Homomorphic Encryption over the Integers (2010) https://eprint.iacr.org/2009/616
* 2nd gen
    * **BGV**(2011), LTV(2012), **BFV**(2012), GHS(2012), BLLN(2013)
* 3rd gen
    * FHEW(2014) TFHE(2016)
* 4th gen
    * **CKKS**(2016)


### Microsoft SEAL
> Microsoft SEAL is an easy-to-use open-source (MIT licensed) homomorphic encryption library...
>Microsoft SEAL comes with two different homomorphic encryption schemes with very different properties. The BFV and BGV schemes allow modular arithmetic to be performed on encrypted integers. The CKKS scheme allows additions and multiplications on encrypted real or complex numbers, but yields only approximate results. In applications such as summing up encrypted real numbers, evaluating machine learning models on encrypted data, or computing distances of encrypted locations CKKS is going to be by far the best choice. For applications where exact values are necessary, the BFV and BGV schemes are more suitable.

### Eksempel
Microsoft prøvde i 2015 å se på edit-disctance på DNA. Her brukes et 4 tegns alfabet, **A**denine , **T**hymine, **G**uanine, **C**ytosine.

https://www.microsoft.com/en-us/research/publication/homomorphic-computation-of-edit-distance/

|(n, m)| Depth | Ring Mod $\Phi_d$ |Key Enc|Total|Amortized|
|-----------|---|-------|-------|-------|-------|
|(1,1)|1|d=4369|256|1.4761s|0.1118s|0.0693s|0.0003s|
|(2,2)|2|d=4369|256|1.8358s|0.2844s|0.2532s|0.0009s|
|(3,3)|8|d=8191|630|7.0162s|1.7117s|34.3091s|0.0544s|
|(4,4)|9|d=8191|630|7.4489s|2.4154s|67.5116s|0.1071s|
|(6,6)|16|d=13981|600|16.1076s|9.9498s|26min|33s|2.6555s|
|(8,8)|19|d=15709|682|27.5454s|16.4524s|4h 50min|25.4366s|
|(50,50)|||||~1 day|

> Currently we could not implement our algorithm for larger parameters [than (8,8)] due to large memory requirements

### FHEAAS
Sør Koreanske ncloud.com gjør det mulig å utføre FHE på data for oss dødelige. Bruker Homomorphic Encryption for Arithmetic of Approximate Numbers (HEaaN) aka Cheon-Kim-Kim-Song CKKS.
![heaan example](heaan-example_extask2_vpc_en.png)


### Microsoft Edge - Password manager
![Edge-password-monitor](Edge-password-monitor.png)
Private Set Intersection (PSI) refers to a functionality where two parties, each holding a private set of items, can check which items they have in common without revealing anything else to each other.

## Dypdykk i CKKS

![Cryptotree_diagrams-2.svg](https://blog.openmined.org/content/images/2020/08/Cryptotree_diagrams-2.svg)
Message $m$ vector (vector med floats) blir encodet til med en *Scaling factor* $\Delta$ til Plaintext .
Plaintext blir kryptert og får lengden $\log Q$ bits

Message $m$ vector (vector med floats) blir encodet til med en *Scaling factor* $\Delta$ til Plaintext .
Plaintext blir kryptert og får lengden $\log Q$ bits

### Multiplisering

* Message $m$ vector (vector med floats) blir encodet til med en *Scaling factor* $\Delta$ til Plaintext .
* Plaintext blir kryptert og får lengden $\log Q$ bits
* $Q = q_0 \times \Delta ^L $
    * $ q_0$: Base modulus
    * $ Q_l = q_0 \times \Delta ^\ell$: Ciphertext modulus
* Level
    * Jeg ser på level $\ell$ som liv i zelda, og når man har 0 liv igjen, kan man ikke f.eks gjøre flere multiplikasjoner.
* Overordnet multiplikasjon.
    * Multiplikasjon: $(ct_a, \ell, \Delta)\times(ct_b,\ell,\Delta) \rightarrow (ct_mul, \ell, \Delta^2)$. Scale øker.
    * Relinearization: $(ct_{mul},\ell,\Delta^2)(ct'_{mul},\ell, \Delta^2)$ Relinearization gjør selve cipherteksten mindre.
    * Rescale: $(ct'_{mul},\ell,\Delta^2)\rightarrow(ct''_mul,\ell-1,\Delta)$

# LPSE **L**ightweight **p**assword-**s**trength estimation for password meters

> Password strength can be measured by comparing the
> similarity between a given password vector and a standard
> strong-password vector.
> We determine the similarity between the two password
> vectors from three aspects:
> * the structure of the password
>     * that is, what kinds of characters compose the password and the pro-portions of various characters
> * the password length
> * the number of insertion, substitution, and deletion operations required to transform a given password into a standard strong password. 

### Passordvektor:

En passordvektor ser slik ut: $\alpha = x_1, x_2, x_3, x_4, x_5$. Der verdiene er hennoldsvis vektorverdi av tall, små bokstaver, store bokstaver, spesialtegn og lengden av passordet.

Table 1 – General rules for mapping characters to
vectors.
|Patterns|Vector value| Example char $\rightarrow$ vector|
|--------|------------|------------------------------|
|Digits|1| 8| $\rightarrow$ 1|
|Lowercase letters| 1| d $\rightarrow$ 1|
|Uppercase letters| 2| G $\rightarrow$ 2|
|Special characters| 3| & $\rightarrow$ 3|
|Two identical characters|Equivalent to one character vector|aa $\rightarrow$ 1, 3a3a $\rightarrow$ 2|
|Two consecutive characters|Equivalent to one character vector|AB $\rightarrow$ 2,1a2b $\rightarrow$ 2|

### En sterk passordvektor $\alpha_{s(ecure)}$
Paperet påstår:
> we believe that a strong password should be randomly generated, and the password length should be greater than 16 characters.

Hvor stor vil en passordvektoren til et tilfeldig passord på 16 tegn være?

La oss ta utgangspunkt i et ASCII keyboard med 96 tegn, 10 tall, 26 små og store bokstaver og 32 spesialtegn. Et tilfledig generert passord burde derfor bestå av:
* $16 \times {10 \over 96} \approx 2$ tall.
* $16 \times {26 \over 96} \approx 5$ små bokstaver.
* $16 \times {26 \over 96} \approx 5$ store bokstaver.
* $16 \times {32 \over 96} \approx 6$ spesialtegn.

... og derfor ha passordvektoren $\alpha_s = \{2,5,10,18,18\}$


### Eksempel
Hvilken vektor vil passordet `aa35*TX1` ha?
* Fjern alle like og bokstaver i rekkefølge. `a35*TX1`
* Hvor mange tall har vi? $x_1=3$
* Hvor mange små bokstaver har vi? $x_2=1$
* Hvor mange store bokstaver har vi? $x_3=2\times2=4$
* Hvor mange tegn har vi? $x_4 = 1 \times 3 = 3$
* Hvor langt er passordet? $x_5 = 8$

$\alpha_{\text{aa35*TX1}}=\{3,1,4,3\}$

### Cosine similarity $cos(\phi)$
$$
\begin{align}
\text{Vectors}\;A,B\\
\text{cosine similarity} &= cos(\phi)\\
 &= {{A \cdot B}\over {||A|| \times ||B||}} \\
 &={ {\sum_{n=1}^n A_i \cdot B_i}\over{\sqrt { \sum_{n=1}^n A_i^2} \cdot \sqrt{ \sum_{n=1}^n A_i^2}}}
\end{align}
$$

### Cosine-length similarity $s_c(\alpha)$

$$
\begin{align}
\text{Passord som testes} &= \alpha\\
\text{sikker passordvektor} &= \alpha_s\\
s_c(\alpha) &= \cos(\phi)\times {{\min(|\alpha||\alpha_s|)} \over {\max(|\alpha||\alpha_s|)}} \\
|\alpha| &= \sqrt{\sum{5}_{i=1}(x_i)^2}
\end{align}
$$

### Password-distane
Dersom Microsoft gikk tom for minne for å se på edit-distance ved et 4-alfabets string på lengde over 8, så ville jeg slitt med et 96-alfabets passord på lengde 18.

Flere regler. Dato, brukernavn, plassering på keyboard. Bytte ut leetspeak. Mest vanlige 2 og 3 bokstavskombinasjoner byttes ut. Gjenstår password-length distance.

# HELPSE **H**ormomorphic Encrypted Lighthweight Password-Strength Estimation for password meters

Notater:
ref: https://eprint.iacr.org/2016/421.pdf
